# 경량화 과제_2
경량화 두번째 과제 파일 입니다.
이번 과제는 Student Network의 학습을 진행해 보겠습니다.
세부 내용은 다음과 같습니다. 

```
과제2.  Student Network의 학습
1. Dataset & Dataloader 만들기 
2. Student Network 정의 하기
3. Student Network 학습시키기

```



In [ ]:
# GPU 할당확인 
import torch

print('CUDA GPU availalbe : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')

## 드라이브 연결 및 PATH 설정 

드라이브 마운트와 PATH설정을 하는 단계입니다. 예제는 아래와 같은 상황에서 돌아가도록 맞춰졌으며, 
자신의 드라이브 환경에 맞추어 설정하시면 되겠습니다. 

```
dataset
  |
  |-  teacher.pth
  |-  경량화 과제_2(문제)
  |-  APY170401435_subset_13gb
```





In [ ]:
# 코랩 내 드라이브와 연결
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Path 설정

import os
if not os.path.exists('/content/gdrive/My Drive/dataset/'):
  os.mkdir('/content/gdrive/My Drive/dataset/')
os.chdir('/content/gdrive/My Drive/dataset/') # Data_Path
current_path = os.getcwd() # current folder
print('current_path', current_path)

current_path /content/gdrive/My Drive/dataset


## 파일이동 확인 
정상적으로 파일이 옮겨졌는지 확인 합니다. 

In [ ]:
from glob import glob

image = sorted(glob('/content/gdrive/My Drive/dataset/total/*.jpg'))
meta = sorted(glob('/content/gdrive/My Drive/dataset/total/*.metadata'))

if len(image) == len(meta):
    print('train data ({} files) unzip succeed'.format(len(image)))
else:
    print('train data unzip failed')
    print('압축을 확인해주세요.')

train data (3881 files) unzip succeed


## 사용자 정의함수 


In [ ]:
# import 
import numpy as np
from PIL import Image
import json
from glob import glob
from pathlib import Path

import cv2
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import time
from datetime import timedelta
from collections import deque, defaultdict
import re 
import matplotlib.image as mpimg
from collections import OrderedDict
from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET 
from tqdm import tqdm

# torch 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

# Dataset / Dataloader


1. Image_Dataset_ : dataset 데이터셋을 정의하는 객체
2. get_loader : Dataloader 객체를 return하는 함수



In [ ]:
class AttributeDict(dict):
    def __init__(self):
        self.__dict__ = self
        

class ConfigTree:
    def __init__(self):
        self.DATASET = AttributeDict()
        self.SYSTEM = AttributeDict()
        self.TRAIN = AttributeDict()
        self.MODEL = AttributeDict()
        self.KD = AttributeDict()

class Image_Dataset_(Dataset):
    def __init__(self, root, cfg=None, transform=None):
        """
        root : './data/'
        """
        super(Image_Dataset_, self).__init__()
        self.root = root
        self.cfg = cfg
        self.transform = transform
        self.images = sorted(glob(self.root + '/*.jpg'))
        self.annotations = sorted(glob(self.root + '/*.metadata'))
        
    def __getitem__(self, idx):
# ---------------------------------------------------------------------------- #
#  1. getitem 완성시키기
# * 주의 * 밑에 사용된 get_gt, label2logit 함수를 이용하여 label 을 만들어야함.
#   1) .metadata를 get_gt로 읽기
#   2) label2logt을 활용하여 1) 값 int 로 변경 
# ---------------------------------------------------------------------------- #

       # 채우기 #
        metadata = self.annotations[idx]
        gt_class = self.get_gt(metadata)
        label = self.label2logit(gt_class)
        image = cv2.imread(self.images[idx], cv2.IMREAD_COLOR)
        image = Image.fromarray(image).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return image, label
        
    def __len__(self):
        assert len(self.images) == len(self.annotations), "# of image files and # of meatadata files do not match"
        return len(self.images)
    
     
# ---------------------------------------------------------------------------- #
#  과제 1 에서 작성한 함수를 적용
# .metadata 를 읽어서 22번째 값(male or female) 을 가져오기 ... 
# ---------------------------------------------------------------------------- #

    def get_gt(self, meta_file):
        gt_class = None
        meta_file = Path(meta_file)
        f = open(meta_file, "r")
        

        # 채우기 #
        idx = 0
        for line in f.readlines():
          if idx == 22:
            gt_class = line.split()[1]
            break
          idx += 1

        return gt_class
    
    def label2logit(self,label):
        str_dict = {
            'female': 0,
            'male':1
        }
        return str_dict[label]

def get_loader(dataset, cfg, shuffle=True):
    return DataLoader(dataset, batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=shuffle)

In [ ]:
# data_root = "./total"

# dataset = Image_Dataset_(data_root)

# for idx in tqdm(range(len(dataset))):
#    _, label = dataset.__getitem__(idx)
#    if label in [0,1]:         
#      pass
#    else:
#      print("Check __getitem__")
#      break

# Image Augmentation 
 get_augmentation : image transform 객체를 return하는 함수
- resize_crop 
- random_flip
- color_jitter


In [ ]:
def get_augmentation(size=224, use_flip=True, use_color_jitter=False, use_gray_scale=False, use_normalize=False):
    resize_crop = transforms.RandomResizedCrop(size=size)
    random_flip = transforms.RandomHorizontalFlip(p=0.5)
    color_jitter = transforms.RandomApply([
        transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)
    ], p=0.8)
    
    gray_scale = transforms.RandomGrayscale(p=0.2)
    normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    to_tensor = transforms.ToTensor()
    
    transforms_array = np.array([resize_crop, random_flip, color_jitter, gray_scale, to_tensor, normalize])
    transforms_mask = np.array([True, use_flip, use_color_jitter, use_gray_scale, True, use_normalize])
    
    transform = transforms.Compose(transforms_array[transforms_mask])
    
    return transform


## Student model

정의 해보기 



In [ ]:
# ---------------------------------------------------------------------------- #
# 1. (코드 채우기) 학습에 사용 될 모델(Student)을 정의해 봅시다. 
# *주의* 마지막 레이어는 2개가 사용되어야 됩니다. 
# ---------------------------------------------------------------------------- #

class Student(nn.Module):
  def __init__(self):
      super().__init__()
      # self.cnn1 = nn.Conv2d(in_channels=3, out_channels=128,kernel_size=3,stride=1,padding=2)
      # self.norm1 = nn.BatchNorm2d(num_features=128)
      # self.relu1 = nn.ReLU()
      # self.drop1 = nn.Dropout2d(p=0.1) 
      # conv_block = nn.Sequential(      
      #     nn.Conv2d(in_channels=128, out_channels=64,kernel_size=3,stride=1,padding=1),
      #     nn.BatchNorm2d(num_features=64),
      #     nn.ReLU(),
      #     nn.Dropout2d(p=0.1)
      #     )
      # self.layer2 = conv_block
      # self.fc = nn.Linear(in_features=226,out_features=2,bias=True)
      self.layer = torchvision.models.resnet34(pretrained=True)
      num_ftrs = self.layer.fc.in_features
      self.layer.fc = nn.Linear(num_ftrs,2, bias=True)

  def forward(self, x):
    # x = self.cnn1(x)
    # x = self.norm1(x)
    # x = self.relu1(x)
    # x = self.drop1(x)
    # x = self.layer2(x)
    # print(x.shape)
    # x = x.view(-1, 226)
    # x = self.fc(x)
    # print(x.shape)
    # x = torch.sigmoid(x)
    # print(x.shape)
    x= self.layer(x)
    return x

In [ ]:
Student()

Student(
  (layer): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [ ]:
# 모델 확인하기 
from torchsummary import summary

try:
  # assert Student().model.fc.out_features ==2
  summary(Student(), (3,224, 224),device='cpu')
except  RuntimeError as err:
  print(err)
  print('Student Network를 재정의 해주세요.')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## Train

- Trainer : config에서 주어진 실험 조건들로 학습을 행하는 객체

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import timedelta
import time
from tqdm import tqdm

class Trainer:
    def __init__(self, model, optimizer, cfg, device, train_loader, val_loader):
        self.cfg = cfg
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.save_path = cfg.BASE_SAVE_DIR
        
        print("[INFO] using gpu {}".format(cfg.SYSTEM.GPU))

        self.model = model.to(device)
        
        update_params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = optimizer(update_params, lr=cfg.TRAIN.BASE_LR, weight_decay=cfg.TRAIN.WEIGHT_DECAY)
        self.criterion = nn.CrossEntropyLoss()
        self.total_step = 0
        self.training_time = 0
        
        print(model)
        print()
        print(optimizer)
        print()
        
    def train_eval(self, name='teacher'):
        print("training start!\n")
        for epoch in tqdm(range(self.cfg.TRAIN.EPOCH)):
            self.train_one_epoch(epoch)
            self.test_one_epoch(epoch)
            self.save_model(epoch,name)
    
    def train_one_epoch(self,epoch):
        print('\nEpoch: %d' % epoch)
        self.model.train()
        
        train_loss = 0
        correct = 0
        total = 0
        for batch_idx, (inputs, targets) in enumerate(self.train_loader):
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % self.cfg.TRAIN.PERIOD ==0:
                print('Loss: %.3f | Acc: %.3f%% ' %(train_loss/(batch_idx+1),100.*correct/total))
        
    def test_one_epoch(self,epoch):
        self.model.eval()
        
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(self.val_loader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                
        print(f'Accuracy of the network on the {total} test images: %d %%' % (
        100 * correct / total))
        
    
    def save_model(self, epoch, name='student'):
        print('saved model {}'.format(self.save_path)) 
        if not os.path.exists(self.save_path):
            os.mkdir(self.save_path)
        file_name = '{}_{}.pth'.format(name, epoch)
        file_path = os.path.join(self.save_path, file_name)
        torch.save(self.model.state_dict(), file_path)
            
        return None      

# Config

모델 변수(즉, 모델의 학습 조건을 설정해주는 공간 ) 

In [ ]:
# config
config = ConfigTree()
config.SYSTEM.GPU = 0 # GPU 번호 
config.BASE_SAVE_DIR = './outputs' # 모델 파라미터가 저장되는 위치

config.DATASET.ROOT = "./total" # 데이터 위치
config.DATASET.NUM_CLASSES = 2 # 분류해야 하는 클래스 종류의 수 
config.DATASET.RATIO = 0.3 # train, test split 비율 




# ---------------------------------------------------------------------------- #
#  밑에 있는 hyper-parameter 값을 조정하여 정확도 올려 보도록 합니다. 
# ---------------------------------------------------------------------------- #

# 사용할 augmentation
config.TRAIN.AUGMENTATION = {'size' : 224,
                             'use_flip' : True,
                             'use_color_jitter' : False,
                             'use_normalize' : True
} 
config.TRAIN.WEIGHT_DECAY = 0.9
config.TRAIN.EPOCH = 30 # 총 학습 에폭 
config.TRAIN.BATCH_SIZE = 32 # 배치 사이즈 
config.TRAIN.BASE_LR = 1e-04 # 러닝 레이트 
config.TRAIN.PERIOD = 1 # loss 측정 주기
config.MODEL.OPTIM = 'SGD' # 'ASGD, Adam, rmsprop etc...'

config.TRAIN.STUDENT_MODEL = Student()

## main 

학습 코드를 실행 

In [ ]:
def main(config):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  data_root = config.DATASET.ROOT
  transform = get_augmentation(**config.TRAIN.AUGMENTATION)
  dataset = Image_Dataset_(data_root, config, transform=transform)

  len_valid_set = int(config.DATASET.RATIO*len(dataset))
  len_train_set = len(dataset) - len_valid_set

  print("The length of Train set is {}".format(len_train_set))
  print("The length of Valid set is {}".format(len_valid_set))

  train_dataset , valid_dataset = torch.utils.data.random_split(dataset ,
                                                                [len_train_set, len_valid_set])

  # shuffle and batch the datasets
  train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=config.TRAIN.BATCH_SIZE,
                                            shuffle=True)
  valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                            batch_size=config.TRAIN.BATCH_SIZE,
                                            shuffle=False)

  # define model(student)
  model = config.TRAIN.STUDENT_MODEL
  optimizer = optim.__dict__[config.MODEL.OPTIM]

  # fitting ... 
  trainer = Trainer(model, optimizer, config, device,
                    train_loader, valid_loader)
  trainer.train_eval()


## Student Network 학습 시키기

In [ ]:
main(config)






  0%|          | 0/30 [00:00<?, ?it/s]

The length of Train set is 2717
The length of Valid set is 1164
[INFO] using gpu 0
Student(
  (layer): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fals






  3%|▎         | 1/30 [17:24<8:25:03, 1044.94s/it]


Epoch: 1
Loss: 0.635 | Acc: 65.625% 
Loss: 0.628 | Acc: 64.062% 
Loss: 0.625 | Acc: 67.708% 
Loss: 0.652 | Acc: 63.281% 
Loss: 0.641 | Acc: 65.625% 
Loss: 0.651 | Acc: 63.021% 
Loss: 0.653 | Acc: 62.946% 
Loss: 0.654 | Acc: 62.109% 
Loss: 0.658 | Acc: 60.764% 
Loss: 0.657 | Acc: 60.312% 
Loss: 0.657 | Acc: 60.227% 
Loss: 0.654 | Acc: 60.677% 
Loss: 0.654 | Acc: 60.577% 
Loss: 0.653 | Acc: 60.268% 
Loss: 0.650 | Acc: 60.833% 
Loss: 0.650 | Acc: 60.547% 
Loss: 0.653 | Acc: 60.662% 
Loss: 0.654 | Acc: 60.417% 
Loss: 0.657 | Acc: 60.033% 
Loss: 0.656 | Acc: 60.000% 
Loss: 0.656 | Acc: 59.673% 
Loss: 0.658 | Acc: 59.091% 
Loss: 0.658 | Acc: 59.103% 
Loss: 0.661 | Acc: 58.984% 
Loss: 0.659 | Acc: 59.375% 
Loss: 0.659 | Acc: 59.375% 
Loss: 0.657 | Acc: 59.259% 
Loss: 0.654 | Acc: 60.045% 
Loss: 0.656 | Acc: 60.129% 
Loss: 0.657 | Acc: 59.688% 
Loss: 0.659 | Acc: 59.476% 
Loss: 0.660 | Acc: 58.984% 
Loss: 0.662 | Acc: 58.617% 
Loss: 0.663 | Acc: 58.272% 
Loss: 0.665 | Acc: 58.214% 
Loss: 0.66






  7%|▋         | 2/30 [25:37<6:50:19, 879.25s/it] 

...






 93%|█████████▎| 28/30 [3:57:33<16:15, 487.72s/it]


Epoch: 28
Loss: 0.378 | Acc: 87.500% 
Loss: 0.391 | Acc: 84.375% 
Loss: 0.409 | Acc: 84.375% 
Loss: 0.423 | Acc: 82.812% 
Loss: 0.426 | Acc: 83.750% 
Loss: 0.431 | Acc: 83.333% 
Loss: 0.430 | Acc: 83.929% 
Loss: 0.428 | Acc: 83.594% 
Loss: 0.428 | Acc: 84.375% 
Loss: 0.429 | Acc: 85.000% 
Loss: 0.432 | Acc: 84.659% 
Loss: 0.429 | Acc: 85.156% 
Loss: 0.429 | Acc: 84.615% 
Loss: 0.436 | Acc: 83.482% 
Loss: 0.434 | Acc: 83.750% 
Loss: 0.438 | Acc: 83.008% 
Loss: 0.439 | Acc: 83.088% 
Loss: 0.442 | Acc: 83.160% 
Loss: 0.440 | Acc: 83.224% 
Loss: 0.438 | Acc: 83.750% 
Loss: 0.439 | Acc: 83.929% 
Loss: 0.436 | Acc: 83.949% 
Loss: 0.437 | Acc: 83.560% 
Loss: 0.438 | Acc: 83.333% 
Loss: 0.438 | Acc: 83.375% 
Loss: 0.438 | Acc: 83.173% 
Loss: 0.437 | Acc: 82.986% 
Loss: 0.440 | Acc: 82.812% 
Loss: 0.442 | Acc: 82.435% 
Loss: 0.443 | Acc: 82.188% 
Loss: 0.443 | Acc: 82.157% 
Loss: 0.443 | Acc: 82.129% 
Loss: 0.442 | Acc: 82.481% 
Loss: 0.442 | Acc: 82.629% 
Loss: 0.442 | Acc: 82.500% 
Loss: 0.4






 97%|█████████▋| 29/30 [4:05:38<08:06, 486.92s/it]


Epoch: 29
Loss: 0.392 | Acc: 81.250% 
Loss: 0.427 | Acc: 85.938% 
Loss: 0.418 | Acc: 85.417% 
Loss: 0.419 | Acc: 83.594% 
Loss: 0.454 | Acc: 81.875% 
Loss: 0.440 | Acc: 82.292% 
Loss: 0.429 | Acc: 83.929% 
Loss: 0.432 | Acc: 83.594% 
Loss: 0.430 | Acc: 84.375% 
Loss: 0.427 | Acc: 84.688% 
Loss: 0.424 | Acc: 85.227% 
Loss: 0.429 | Acc: 84.115% 
Loss: 0.429 | Acc: 84.615% 
Loss: 0.431 | Acc: 83.705% 
Loss: 0.429 | Acc: 83.958% 
Loss: 0.424 | Acc: 84.180% 
Loss: 0.424 | Acc: 84.007% 
Loss: 0.428 | Acc: 83.854% 
Loss: 0.432 | Acc: 83.717% 
Loss: 0.428 | Acc: 84.375% 
Loss: 0.427 | Acc: 84.375% 
Loss: 0.428 | Acc: 84.233% 
Loss: 0.428 | Acc: 84.375% 
Loss: 0.427 | Acc: 84.505% 
Loss: 0.428 | Acc: 84.500% 
Loss: 0.424 | Acc: 84.976% 
Loss: 0.425 | Acc: 84.954% 
Loss: 0.423 | Acc: 85.491% 
Loss: 0.422 | Acc: 85.560% 
Loss: 0.422 | Acc: 85.312% 
Loss: 0.420 | Acc: 85.484% 
Loss: 0.421 | Acc: 85.156% 
Loss: 0.420 | Acc: 85.417% 
Loss: 0.422 | Acc: 85.294% 
Loss: 0.421 | Acc: 85.357% 
Loss: 0.4






100%|██████████| 30/30 [4:13:44<00:00, 507.48s/it]
